In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
import collections
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm

In [50]:
input_text = au.read_txt_file_lines('input.txt')
n_rows = len(input_text)
n_cols = len(input_text[0])
for ii in range(1, n_rows):
    assert len(input_text[ii]) == n_cols, f'row {ii} has {len(input_text[ii])} cols, not {n_cols}'
print(f'input has {n_rows} rows and {n_cols} cols')

for ic, col in enumerate(input_text[0]):
    if ic != 1:
        assert col == '#', ic
for ic, col in enumerate(input_text[-1]):
    if ic != n_cols - 2:
        assert col == '#', ic
for row in input_text:
    assert row[0] == '#', f'row {row} not valid'
    assert row[-1] == '#', f'row {row} not valid'

unique_chars = set()
for row in input_text:
    unique_chars.update(row)
print(f'unique_chars: {unique_chars}')

input has 141 rows and 141 cols
unique_chars: {'>', 'v', '#', '.'}


In [53]:
path_tuple = collections.namedtuple('path', 'head visited_nodes')

def progress_til_junction(path, input_text, end_node=(n_rows - 1, n_cols - 2)):
    n_rows, n_cols = len(input_text), len(input_text[0])
    current_row, current_col = path.head
    while True:
        possible_next_nodes = []
        if input_text[current_row][current_col] == '>':
            if current_col < n_cols - 1 and input_text[current_row][current_col + 1] in ['.', '>']:
                possible_next_nodes.append((current_row, current_col + 1))
        elif input_text[current_row][current_col] == 'v':
            if current_row < n_rows - 1 and input_text[current_row + 1][current_col] in ['.', 'v']:
                possible_next_nodes.append((current_row + 1, current_col))
        else:
            if current_col < n_cols - 1 and input_text[current_row][current_col + 1] in ['.', '>']:
                possible_next_nodes.append((current_row, current_col + 1))
            if current_col > 0 and input_text[current_row][current_col - 1] == '.':
                possible_next_nodes.append((current_row, current_col - 1))
            if current_row < n_rows - 1 and input_text[current_row + 1][current_col] in ['.', 'v']:
                possible_next_nodes.append((current_row + 1, current_col))
            if current_row > 0 and input_text[current_row - 1][current_col] == '.':
                possible_next_nodes.append((current_row - 1, current_col))

        possible_next_nodes = [node for node in possible_next_nodes if node not in path.visited_nodes]

        if len(possible_next_nodes) == 0:
            if path.head == end_node:
                return True, path
            else:
                print(f'path {path} has no possible next nodes')
                return False, []
        elif len(possible_next_nodes) == 1:
            new_visited_nodes = copy.deepcopy(path.visited_nodes)
            new_visited_nodes.add(path.head)
            path = path._replace(head=possible_next_nodes[0])
            path = path._replace(visited_nodes=new_visited_nodes)
            current_row, current_col = path.head
        else:
            new_paths = []
            for node in possible_next_nodes:
                new_visited_nodes = copy.deepcopy(path.visited_nodes)
                new_visited_nodes.add(path.head)
                new_paths.append(path_tuple(node, new_visited_nodes))
            return False, new_paths 
        
def find_longest_path(input_text, start_row=0, start_col=1, verbose=1):
    visited_nodes = set()
    visited_nodes.add((start_row, start_col))
    queue_paths = collections.deque()
    queue_paths.append(path_tuple((start_row, start_col), visited_nodes))
    paths_to_end = []
    i_path = 0
    while len(queue_paths) > 0: 
        i_path += 1
        current_path = queue_paths.popleft()
        if verbose > 0:
            print(f'starting new path (#{i_path}), starting at {current_path.head}. {len(queue_paths)} paths left in queue')
        reached_end, new_path = progress_til_junction(current_path, input_text)
        if reached_end:
            if  verbose > 1:
                print(f'found end {new_path}')
            paths_to_end.append(new_path)
        else:
            if verbose > 1:
                print(f'adding {len(new_path)} new paths')
            queue_paths.extend(new_path)

    if verbose > 0:
        print(f'found {len(paths_to_end)} paths to end')
    return paths_to_end

In [ ]:
paths_to_end = find_longest_path(input_text, verbose=1)
paths_length = [len(path.visited_nodes) for path in paths_to_end]
max_path_length = max(paths_length)
print(f'max_path_length: {max_path_length}')

In [42]:
path((0, 1), set([(0, 1)]))

path(head=(0, 1), visited_nodes={(0, 1)})